<a href="https://colab.research.google.com/github/Dusein/MachineLearningTask/blob/main/14thWeekTask/Bidirectional_RNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Libraries and Load Dataset
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [5]:
# Ganti 'your_file_path' dengan path sebenarnya ke file CSV Anda di Google Drive
file_path = '/content/sample_data/diabetes_012_health_indicators_BRFSS2015.csv'

# Membaca file CSV ke dalam DataFrame
data = pd.read_csv(file_path)

# Menampilkan beberapa baris pertama dari DataFrame
print(data.head())

   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1           0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2           0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3           0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4           0.0     1.0       1.0        1.0  24.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           0.0     0.0  ...            1.0   
1                   0.0           1.0     0.0  ...            0.0   
2                   0.0           0.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0      5.0      18.0      15.0       1.0  0.0   9.0        4.0   
1     

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

In [8]:
# Pisahkan fitur dan label
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

# Standarisasi data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data ke Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader untuk batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 karena bidirectional

    def forward(self, x):
        # Inisialisasi hidden state
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Ambil output terakhir
        out = self.fc(out)
        return out


In [14]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, early_stopping_patience=10):
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            # Tambahkan dimensi baru untuk sequence_length
            X_batch, y_batch = X_batch.unsqueeze(1).to(device), y_batch.to(device)  # [batch_size, 1, input_size]
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Evaluasi pada validation set
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # Tambahkan dimensi baru untuk sequence_length
                X_batch, y_batch = X_batch.unsqueeze(1).to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping triggered")
                break


In [16]:
input_size = X_train.shape[1]  # Jumlah fitur (input size)
output_size = len(np.unique(y_train))  # Jumlah kelas (output size)

hidden_sizes = [16, 32, 64]
results_hidden_size = {}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for hidden_size in hidden_sizes:
    print(f"Training with Hidden Size: {hidden_size}")
    model = BiRNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train model
    train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=10)
    results_hidden_size[hidden_size] = model


Training with Hidden Size: 16
Epoch 1/10, Train Loss: 0.4079, Val Loss: 0.3939
Epoch 2/10, Train Loss: 0.3957, Val Loss: 0.3922
Epoch 3/10, Train Loss: 0.3948, Val Loss: 0.3911
Epoch 4/10, Train Loss: 0.3943, Val Loss: 0.3910
Epoch 5/10, Train Loss: 0.3939, Val Loss: 0.3908
Epoch 6/10, Train Loss: 0.3936, Val Loss: 0.3932
Epoch 7/10, Train Loss: 0.3934, Val Loss: 0.3913
Epoch 8/10, Train Loss: 0.3931, Val Loss: 0.3911
Epoch 9/10, Train Loss: 0.3929, Val Loss: 0.3910
Epoch 10/10, Train Loss: 0.3927, Val Loss: 0.3924
Training with Hidden Size: 32
Epoch 1/10, Train Loss: 0.4048, Val Loss: 0.3955
Epoch 2/10, Train Loss: 0.3967, Val Loss: 0.3922
Epoch 3/10, Train Loss: 0.3955, Val Loss: 0.3912
Epoch 4/10, Train Loss: 0.3949, Val Loss: 0.3923
Epoch 5/10, Train Loss: 0.3943, Val Loss: 0.3933
Epoch 6/10, Train Loss: 0.3940, Val Loss: 0.3915
Epoch 7/10, Train Loss: 0.3937, Val Loss: 0.3918
Epoch 8/10, Train Loss: 0.3935, Val Loss: 0.3948
Epoch 9/10, Train Loss: 0.3933, Val Loss: 0.3914
Epoch 10

In [17]:
class BiRNNWithPooling(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling="max", num_layers=1):
        super(BiRNNWithPooling, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 karena bidirectional
        self.pooling = pooling

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)

        if self.pooling == "max":
            out, _ = torch.max(out, dim=1)
        elif self.pooling == "avg":
            out = torch.mean(out, dim=1)

        out = self.fc(out)
        return out


In [18]:
optimizers = {
    "SGD": torch.optim.SGD,
    "RMSProp": torch.optim.RMSprop,
    "Adam": torch.optim.Adam,
}

for opt_name, opt_func in optimizers.items():
    print(f"Training with Optimizer: {opt_name}")
    model = BiRNN(input_size=input_size, hidden_size=32, output_size=output_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = opt_func(model.parameters(), lr=0.001)

    train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=10)


Training with Optimizer: SGD
Epoch 1/10, Train Loss: 0.5030, Val Loss: 0.4095
Epoch 2/10, Train Loss: 0.4069, Val Loss: 0.4008
Epoch 3/10, Train Loss: 0.4029, Val Loss: 0.3991
Epoch 4/10, Train Loss: 0.4018, Val Loss: 0.3985
Epoch 5/10, Train Loss: 0.4012, Val Loss: 0.3981
Epoch 6/10, Train Loss: 0.4008, Val Loss: 0.3978
Epoch 7/10, Train Loss: 0.4005, Val Loss: 0.3975
Epoch 8/10, Train Loss: 0.4003, Val Loss: 0.3973
Epoch 9/10, Train Loss: 0.4000, Val Loss: 0.3971
Epoch 10/10, Train Loss: 0.3998, Val Loss: 0.3970
Training with Optimizer: RMSProp
Epoch 1/10, Train Loss: 0.4018, Val Loss: 0.3956
Epoch 2/10, Train Loss: 0.3965, Val Loss: 0.3923
Epoch 3/10, Train Loss: 0.3952, Val Loss: 0.3920
Epoch 4/10, Train Loss: 0.3948, Val Loss: 0.3948
Epoch 5/10, Train Loss: 0.3946, Val Loss: 0.3935
Epoch 6/10, Train Loss: 0.3941, Val Loss: 0.3909
Epoch 7/10, Train Loss: 0.3939, Val Loss: 0.3914
Epoch 8/10, Train Loss: 0.3937, Val Loss: 0.3922
Epoch 9/10, Train Loss: 0.3936, Val Loss: 0.3910
Epoch 

In [19]:
epochs = [5, 50]

for num_epochs in epochs:
    print(f"Training with Epochs: {num_epochs}")
    model = BiRNN(input_size=input_size, hidden_size=32, output_size=output_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=num_epochs)


Training with Epochs: 5
Epoch 1/5, Train Loss: 0.4044, Val Loss: 0.3939
Epoch 2/5, Train Loss: 0.3963, Val Loss: 0.3951
Epoch 3/5, Train Loss: 0.3953, Val Loss: 0.3920
Epoch 4/5, Train Loss: 0.3946, Val Loss: 0.3936
Epoch 5/5, Train Loss: 0.3944, Val Loss: 0.3916
Training with Epochs: 50
Epoch 1/50, Train Loss: 0.4054, Val Loss: 0.3950
Epoch 2/50, Train Loss: 0.3962, Val Loss: 0.3941
Epoch 3/50, Train Loss: 0.3953, Val Loss: 0.3927
Epoch 4/50, Train Loss: 0.3948, Val Loss: 0.3911
Epoch 5/50, Train Loss: 0.3943, Val Loss: 0.3916
Epoch 6/50, Train Loss: 0.3938, Val Loss: 0.3916
Epoch 7/50, Train Loss: 0.3937, Val Loss: 0.3900
Epoch 8/50, Train Loss: 0.3935, Val Loss: 0.3921
Epoch 9/50, Train Loss: 0.3932, Val Loss: 0.3914
Epoch 10/50, Train Loss: 0.3932, Val Loss: 0.3906
Epoch 11/50, Train Loss: 0.3929, Val Loss: 0.3928
Epoch 12/50, Train Loss: 0.3927, Val Loss: 0.3940
Epoch 13/50, Train Loss: 0.3927, Val Loss: 0.3910
Epoch 14/50, Train Loss: 0.3924, Val Loss: 0.3916
Epoch 15/50, Train L